# 한국 맛집 어플 데이터를 크롤링 후 학습

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import os


rating_file_path = os.getenv('HOME') +'/aiffel/H3/foodrating/app_db2.csv'
#ratings_cols = ['ProductId', 'ProfileName', 'Score', 'Summary']
data1 = pd.read_csv(rating_file_path, header = None, index_col = None)
data1.columns = ['userId','Menu','Rating','매운/안매운','한중일양','뜨거운/차가운','육해공', '식사/디저트']
data1.head()
#data1.tail()

,userId,Menu,Rating,매운/안매운,한중일양,뜨거운/차가운,육해공,식사/디저트
0,배고파,치킨,4,안매운거,한식,뜨거운거,공,식사
1,배고파,마라탕,5,매운거,중식,뜨거운거,육,식사
2,배고파,떡볶이,5,매운거,한식,뜨거운거,육,식사
3,배고파,피자,5,안매운거,양식,뜨거운거,육,식사
4,배고파,찜닭,5,안매운거,한식,뜨거운거,공,식사


In [2]:
data1["userId"].nunique()

60

# 크게 두가지 모델로 나눌수 있습니다 : 
# 1) 아이템 기반 추천시스템 2) 유저 기반 추천시스템

# 유저기반 추천시스템

In [3]:
user_df = pd.read_csv(rating_file_path, header = None, index_col = None)
user_df.columns = ['userId','Menu','Rating','hot','kcjw','hc','csa', 'md']
user_df.head()

,userId,Menu,Rating,hot,kcjw,hc,csa,md
0,배고파,치킨,4,안매운거,한식,뜨거운거,공,식사
1,배고파,마라탕,5,매운거,중식,뜨거운거,육,식사
2,배고파,떡볶이,5,매운거,한식,뜨거운거,육,식사
3,배고파,피자,5,안매운거,양식,뜨거운거,육,식사
4,배고파,찜닭,5,안매운거,한식,뜨거운거,공,식사


In [4]:
user_df["userId"].nunique()

60

# 인기 많은 음식들 (top 30)

In [5]:
product_count = user_df.groupby('Menu')['userId'].count()
product_count.sort_values(ascending = False).head(30)

Menu
치킨      35
피자      29
떡볶이     23
돈까스     19
족발      16
회       14
짬뽕      14
파스타     11
김밥      11
삼겹살     11
샐러드     10
탕수육     10
보쌈      10
찜닭      10
김치찌개     8
샌드위치     8
초밥       8
햄버거      8
스테이크     7
볶음밥      7
마라탕      7
닭도리탕     7
해장국      6
김치찜      6
짜장면      6
라멘       5
카레       5
쌀국수      5
와플       5
닭갈비      4
Name: userId, dtype: int64

# 3. 내가 선호하는 식품 7가지 골라서 rating 에 추가

In [6]:

my_favorite = ['치킨', '햄버거', '족발','회', '돼지김치찜', '삼겹살', '돈까스']

my_foodlist = pd.DataFrame({'userId': ['규환']*7,  'Rating':[5]*7,
                           'Menu': ['치킨', '햄버거', '족발','회', '돼지김치찜', '삼겹살', '돈까스'],
                          'hot' : ['안매운거', '안매운거', '안매운거', '안매운거', '매운거', '안매운거', '안매운거'],
                            'kcjw' : ['양식', '양식', '한식', '일식', '한식', '한식', '일식'],
                            'hc' : ['뜨거운거', '뜨거운거', '뜨거운거', '차가운거', '뜨거운거', '뜨거운거', '뜨거운거'],
                            'csa' : ['공', '육', '육', '해', '육', '육', '육'],
                            'md' : ['식사', '식사', '식사', '식사', '식사', '식사', '식사']})

if not user_df.isin({'userId': ['규환']})['userId'].any():
    user_df = user_df.append(my_foodlist)
user_df.tail(10)

,userId,Menu,Rating,hot,kcjw,hc,csa,md
505,cdou,마파두부밥,3,매운거,중식,뜨거운거,육,식사
506,cdou,족발,3,안매운거,한식,뜨거운거,육,식사
507,cdou,된장찌개,4,안매운거,한식,뜨거운거,육,식사
0,규환,치킨,5,안매운거,양식,뜨거운거,공,식사
1,규환,햄버거,5,안매운거,양식,뜨거운거,육,식사
2,규환,족발,5,안매운거,한식,뜨거운거,육,식사
3,규환,회,5,안매운거,일식,차가운거,해,식사
4,규환,돼지김치찜,5,매운거,한식,뜨거운거,육,식사
5,규환,삼겹살,5,안매운거,한식,뜨거운거,육,식사
6,규환,돈까스,5,안매운거,일식,뜨거운거,육,식사


In [7]:
# 고유한 유저, 메뉴, 등등 찾기
user_unique = user_df['userId'].unique()
menu_unique = user_df['Menu'].unique()
hot_unique = user_df['hot'].unique()
kcpw_unique = user_df['kcjw'].unique()
hc_unique = user_df['hc'].unique()
csa_unique = user_df['csa'].unique()
md_unique = user_df['md'].unique()


# 유저, 메뉴와 다른 컬럼들을 indexing 하는 코드 idx는 index의 약자입니다.

user_to_idx = {v:k for k,v in enumerate(user_unique)}
menu_to_idx = {v:k for k,v in enumerate(menu_unique)}
hot_to_idx = {v:k for k,v in enumerate(hot_unique)}
kcjw_to_idx = {v:k for k,v in enumerate(kcpw_unique)}
hc_to_idx = {v:k for k,v in enumerate(hc_unique)}
csa_to_idx = {v:k for k,v in enumerate(csa_unique )}
md_to_idx = {v:k for k,v in enumerate(md_unique)}


print(user_to_idx['규환'])

60


In [8]:
# CSR Matrix 를 직접 만들어 보기


# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 UserId 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 

temp_user_data = user_df['userId'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(user_df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('유저 ID 컬럼 인덱싱 완료!!')
    user_df['userId'] = temp_user_data   # data['UserId']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('유저 ID 컬럼 인덱싱 실패')

# Menu 컬럼도 동일한 방식으로 인덱싱해 줍니다.

temp_menu_data = user_df['Menu'].map(menu_to_idx.get).dropna()
if len(temp_menu_data) == len(user_df):
    print('메뉴 컬럼 인덱싱 완료!!')
    user_df['Menu'] = temp_menu_data
else:
    print('메뉴 컬럼 인덱싱 실패!!')
    
# 매운/안매운 컬럼도 동일한 방식으로 인덱싱해 줍니다.

temp_hot_data = user_df['hot'].map(hot_to_idx.get).dropna()
if len(temp_hot_data) == len(user_df):
    print('매운안매운 컬럼 인덱싱 완료!!')
    user_df['hot'] = temp_hot_data
else:
    print('매운/안매운 컬럼 인덱싱 실패!!')
    
# 한중일양 컬럼도 동일한 방식으로 인덱싱해 줍니다.

temp_kcjw_data = user_df['kcjw'].map(kcjw_to_idx.get).dropna()
if len(temp_kcjw_data) == len(user_df):
    print('한중일양 컬럼 인덱싱 완료!!')
    user_df['kcjw'] = temp_kcjw_data
else:
    print('한중일양 컬럼 인덱싱 실패!!')
    
# 뜨거운/차가운 컬럼도 동일한 방식으로 인덱싱해 줍니다.

temp_hc_data = user_df['hc'].map(hc_to_idx.get).dropna()
if len(temp_hc_data) == len(user_df):
    print('뜨거운/차가운 컬럼 인덱싱 완료!!')
    user_df['hc'] = temp_hc_data
else:
    print('뜨거운/차가운 컬럼 인덱싱 실패!!')
    
# 육해공 컬럼도 동일한 방식으로 인덱싱해 줍니다.    

temp_csa_data = user_df['csa'].map(csa_to_idx.get).dropna()
if len(temp_csa_data) == len(user_df):
    print('육해공 컬럼 인덱싱 완료!!')
    user_df['csa'] = temp_csa_data
else:
    print('육해공 컬럼 인덱싱 실패!!')


# 식사/디저트 컬럼도 동일한 방식으로 인덱싱해 줍니다.

temp_md_data = user_df['md'].map(md_to_idx.get).dropna()
if len(temp_md_data) == len(user_df):
    print('식사/디저트 컬럼 인덱싱 완료!!')
    user_df['md'] = temp_md_data
else:
    print('식사/디저트 컬럼 인덱싱 실패!!')

user_df

유저 ID 컬럼 인덱싱 완료!!
메뉴 컬럼 인덱싱 완료!!
매운안매운 컬럼 인덱싱 완료!!
한중일양 컬럼 인덱싱 완료!!
뜨거운/차가운 컬럼 인덱싱 완료!!
육해공 컬럼 인덱싱 완료!!
식사/디저트 컬럼 인덱싱 완료!!


,userId,Menu,Rating,hot,kcjw,hc,csa,md
0,0,0,4,0,0,0,0,0
1,0,1,5,1,1,0,1,0
2,0,2,5,1,0,0,1,0
3,0,3,5,0,2,0,1,0
4,0,4,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...
2,60,29,5,0,0,0,1,0
3,60,16,5,0,3,1,2,0
4,60,140,5,1,0,0,1,0
5,60,25,5,0,0,0,1,0


# Compressed Sparse Row Matrix

유저 x 아이템 평가행렬 -> 어림잡아도 엄청난 메모리가 필요하다 (600GB 이상)

유저가 좋아하지 않는 아이템에 대한 정보까지 모두 행렬에 포함되어 계산되기 때문.

평가행렬 내의 대부분의 공간은 0으로 채워짐. 이를 Sparse Matrix 라고 부름.

이런 메모리 낭비를 최소화 하기 위해 유저가 좋아하는 아이템에 대해서만 정보만을 저장하면서

전체 행렬 형태를 유추할 수있는 데이터 구조가 필요

# Sparse Matrix 에 관한 URL :

1) https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005

2)https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix

In [9]:
#CSR MATRIX 만들기
from scipy.sparse import csr_matrix

num_user = user_df['userId'].nunique()
num_product = user_df['Menu'].nunique()

csr_data = csr_matrix((user_df['Rating'], (user_df.userId, user_df.Menu)), shape= (num_user, num_product))
csr_data

<61x141 sparse matrix of type '<class 'numpy.longlong'>'
	with 511 stored elements in Compressed Sparse Row format>

# Matrix Factorization 모델 학습

implict 라는 패키지를 이용

als(AlternatingLeastSqaures) 모델을 사용. Matrix Factorization 에서 쪼개진 두 Feature Matrix 를 한꺼번에 훈련하는것은 잘 수렴하지 않기 때문에,
한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 ALS 방식이 효과적임

In [10]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

#implict 에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREAD'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

#implict AlternatingLeastSquares 모델의 선언


#ALS 클래스의 __init__ 파라미터 살펴보기
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할것인지
# 2. regularization : 과적합 방지하기 위해 정규화 값을 얼마나 사용할 것인지
# 3. use_gpu : GPU 를 사용할 것잉ㄴ지
# 4. iterations : epoch 과 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지
als_model = AlternatingLeastSquares(factors = 100, regularization = 0.01, use_gpu = False, iterations = 50,
                                   dtype = np.float32)

#als 모델은 input 으로 (item X user 꼴의 matrix 를 받기 때문에 Transpose 해줍니다.)

csr_data_transpose = csr_data.T
csr_data_transpose

<141x61 sparse matrix of type '<class 'numpy.longlong'>'
	with 511 stored elements in Compressed Sparse Column format>

In [11]:
als_model.fit(csr_data_transpose)

In [12]:
print(user_to_idx['규환'])

60


# 4.  내가 선호하는 7가지 음식 중 하나와 그 외의 음식 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보기

In [13]:
규환, 족발 = user_to_idx['규환'], menu_to_idx['족발']
규환_vector, 족발_vector = als_model.user_factors[규환], als_model.item_factors[족발]

In [14]:
규환_vector

array([-0.04988303, -0.97294784,  0.7417746 , -0.06597493, -0.8480485 ,
        0.2770032 , -0.65834415,  0.05350488,  0.7896784 , -0.5491745 ,
        0.21918438,  0.45673203,  1.0041324 , -0.00427986,  0.6144158 ,
        0.7416637 , -0.89973533,  0.11885598,  0.59156084, -0.59356916,
        0.22760814, -0.35349783, -0.2749667 ,  0.47743607,  0.07238533,
        0.24162053, -0.69553715,  0.85889524, -0.13126951, -0.24931765,
       -0.06408739,  0.32867214, -0.4000444 ,  1.021529  ,  0.45371735,
        0.6678331 ,  0.6217678 ,  0.3040627 ,  0.8245889 ,  0.21717633,
        0.4781295 ,  0.02025321,  0.58938533,  0.75591964, -0.02870671,
       -0.10562222,  0.25432846,  0.26305985,  0.32819518,  0.30601114,
        0.6301064 ,  0.11109436,  0.44301718, -0.541905  ,  0.5674946 ,
       -0.0437074 ,  0.28147632,  0.27773294,  1.0840454 ,  0.11436753,
        0.39293608,  1.2677531 ,  1.2854066 ,  0.87004566,  0.44689286,
        0.47234207, -0.08998771, -0.05562425, -0.45236447,  0.04

In [15]:
족발_vector

array([-0.03463773, -0.06024525, -0.01148889, -0.05532179, -0.08133009,
        0.03682017,  0.03511563, -0.02877827,  0.03860339,  0.03483708,
       -0.00287245, -0.03064307,  0.10322655,  0.07325795,  0.00287348,
       -0.01106869, -0.00648437,  0.01810774,  0.02920954,  0.00630822,
       -0.01299893, -0.02074266, -0.0579687 , -0.03651355, -0.01690398,
        0.05822389, -0.0098803 ,  0.02535563, -0.0525874 ,  0.0506843 ,
        0.00801479,  0.00538572,  0.01974047,  0.09875428,  0.00670617,
        0.0632026 ,  0.01377836, -0.03172817,  0.02121178, -0.00153258,
       -0.06032758, -0.03768677,  0.03596446,  0.05042633,  0.04913162,
       -0.05344287, -0.08576252,  0.04779577,  0.06852905, -0.03039498,
       -0.0182733 ,  0.06375173,  0.04739819, -0.01992701,  0.00187084,
        0.05462245,  0.09558015, -0.03992235,  0.0307112 , -0.06405053,
       -0.00491135, -0.0299236 ,  0.09217157,  0.03496151, -0.01989432,
        0.0496227 , -0.00203549, -0.03404492,  0.04969214,  0.03

In [16]:
# 규환과 족발이 내적하는 코드

np.dot(규환_vector, 족발_vector)

0.9992406

In [17]:
# 나의 보쌈에 대한 선호도는 어떻게 예측할지

보쌈 = menu_to_idx['보쌈']
보쌈_vector = als_model.item_factors[보쌈]
np.dot(규환_vector, 보쌈_vector)

0.006596901

# 5. 내가 좋아하는 음식과 비슷한 음식을 추천받아 봅시다.

In [18]:
# 삼결살과 비슷한 음식 추천받기

favorite_food = '삼겹살'
menu_id = menu_to_idx[favorite_food]

similar_food = als_model.similar_items(menu_id, N=15)
similar_food

[(25, 0.99999994),
 (65, 0.18490104),
 (24, 0.15046541),
 (49, 0.13174212),
 (91, 0.12910792),
 (93, 0.12871039),
 (92, 0.1285811),
 (129, 0.122039445),
 (128, 0.121951796),
 (30, 0.11438712),
 (140, 0.1138959),
 (19, 0.108399354),
 (20, 0.10837809),
 (113, 0.106520325),
 (15, 0.1030626)]

In [19]:
#음식 이름으로 변경해주기

idx_to_menu = {v:k for k,v in menu_to_idx.items()}
[idx_to_menu[i[0]]for i in similar_food]

['삼겹살',
 '순대국밥',
 '탕수육',
 '술국',
 '설렁탕',
 '소고기',
 '온모밀',
 '곱창',
 '불고기덮밥',
 '쌀국수',
 '돼지김치찜',
 '꽈배기',
 '순대',
 '커피',
 '라멘']

# 6. 내가 좋아할만한 음식들을 추천받아보기

In [20]:
# 음식 추천받기

user = user_to_idx['규환']

#recommend 에서는 user * item CSR Matrix 를 받습니다.

food_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items = True)
food_recommended

[(27, 0.0065968707),
 (3, 0.005114034),
 (64, 0.0043675154),
 (7, 0.0039099045),
 (46, 0.003688354),
 (48, 0.0032781586),
 (30, 0.0030823797),
 (15, 0.0029973462),
 (22, 0.002934534),
 (51, 0.0027536303),
 (77, 0.0024116188),
 (36, 0.0023891777),
 (24, 0.0023376346),
 (39, 0.0022463202),
 (123, 0.0022410713),
 (66, 0.0020771138),
 (119, 0.0020565987),
 (83, 0.0020416863),
 (50, 0.0018189624),
 (56, 0.0017285943)]

In [21]:
[idx_to_menu[i[0]]for i in food_recommended]

['보쌈',
 '피자',
 '초밥',
 '김치찌개',
 '부대찌개',
 '순대국',
 '쌀국수',
 '라멘',
 '팥빙수',
 '치킨마요덥밥',
 '육회',
 '김밥',
 '탕수육',
 '와플',
 '롤',
 '비빕밥',
 '칼국수',
 '비빔냉면',
 '리조또',
 '김치찜']

In [22]:
#이 추천에 기여한 정도 (돈까스의 경우)

돈까스 = menu_to_idx['돈까스']
explain = als_model.explain(user, csr_data, itemid = 돈까스)

[(idx_to_menu[i[0]], i[1]) for i in explain [1]]

[('돈까스', 0.9661983567394195),
 ('돼지김치찜', 0.02021657580770362),
 ('치킨', 0.0041820584557017045),
 ('족발', 0.003949567434646764),
 ('삼겹살', 0.0018769458136590521),
 ('회', -0.0023612825813514107),
 ('햄버거', -0.00387488025413899)]